In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
import pickle

# 1. Load data

olist_customers_dataset

In [ ]:
df_customer = pd.read_csv('./dataset/olist_customers_dataset.csv')

In [ ]:
df_customer.head()

In [ ]:
df_customer.info()

olist_geolocation_dataset

In [ ]:
df_geo = pd.read_csv('./dataset/olist_geolocation_dataset.csv')

In [ ]:
df_geo.head()

In [ ]:
df_geo.info()

olist_order_items_dataset

In [ ]:
df_order_item = pd.read_csv('./dataset/olist_order_items_dataset.csv')

In [ ]:
df_order_item.head()

In [ ]:
df_order_item.info()

In [ ]:
df_order_item['shipping_limit_date'] = pd.to_datetime(df_order_item['shipping_limit_date'])

olist_order_payments_dataset

In [ ]:
df_order_payment = pd.read_csv('./dataset/olist_order_payments_dataset.csv')

In [ ]:
df_order_payment.head()

In [ ]:
df_order_payment.info()

olist_order_reviews_dataset

In [ ]:
df_order_review = pd.read_csv('./dataset/olist_order_reviews_dataset.csv')

In [ ]:
df_order_review.head()

In [ ]:
df_order_review.info()

In [ ]:
df_order_review['review_creation_date'] = pd.to_datetime(df_order_review['review_creation_date'])
df_order_review['review_answer_timestamp'] = pd.to_datetime(df_order_review['review_answer_timestamp'])

olist_orders_dataset

In [ ]:
df_order = pd.read_csv('./dataset/olist_orders_dataset.csv')

In [ ]:
df_order.head()

In [ ]:
df_order.info()

In [ ]:
df_order['order_purchase_timestamp'] = pd.to_datetime(df_order['order_purchase_timestamp'])
df_order['order_approved_at'] = pd.to_datetime(df_order['order_approved_at'])
df_order['order_delivered_carrier_date'] = pd.to_datetime(df_order['order_delivered_carrier_date'])
df_order['order_delivered_customer_date'] = pd.to_datetime(df_order['order_delivered_customer_date'])
df_order['order_estimated_delivery_date'] = pd.to_datetime(df_order['order_estimated_delivery_date'])

olist_products_dataset

In [ ]:
df_product = pd.read_csv('./dataset/olist_products_dataset.csv')

In [ ]:
df_product.head()

In [ ]:
df_product.info()

olist_sellers_dataset

In [ ]:
df_seller = pd.read_csv('./dataset/olist_sellers_dataset.csv')

In [ ]:
df_seller.head()

In [ ]:
df_seller.info()

product_category_name_translation

In [ ]:
df_cat_name = pd.read_csv('./dataset/product_category_name_translation.csv')

In [ ]:
df_cat_name.head()

In [ ]:
df_cat_name.info()

olist_age

In [ ]:
df_age = pd.read_csv('./dataset/olist_age.csv')

In [ ]:
df_age.head()

In [ ]:
df_age.info()

# 2. Feature engineering

1. customer_id

In [ ]:
df_customer_unique_id = duckdb.query("""
select distinct customer_unique_id
from df_customer
""").to_df()

In [ ]:
df_customer_unique_id.head()

2. age

In [ ]:
df_customer_age = duckdb.query("""
select b.customer_unique_id
, min(a.age) age
from df_age a
left join df_customer b
on a.customer_id = b.customer_id
group by b.customer_unique_id
""").to_df()

In [ ]:
df_customer_age.head()

3. customer address

In [ ]:
df_customer_address = duckdb.query("""
select customer_unique_id
, case when customer_state in ('AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO') then 1 else 0 end address_north
, case when customer_state in ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE') then 1 else 0 end address_northeast
, case when customer_state in ('DF', 'GO', 'MT', 'MS') then 1 else 0 end address_centralwest
, case when customer_state in ('ES', 'MG', 'RJ', 'SP') then 1 else 0 end address_southeast
, case when customer_state in ('PR', 'RS', 'SC') then 1 else 0 end address_south
from
    (
    select customer_unique_id, customer_state, customer_city, customer_zip_code_prefix
    , row_number() over(partition by customer_unique_id order by customer_zip_code_prefix) rn
    from df_customer
    ) a
where rn = 1
""").to_df()

In [ ]:
df_customer_address.head()

4. recency

In [ ]:
duckdb.query("""
select max(a.order_purchase_timestamp)
from df_order a
left join df_customer b on a.customer_id = b.customer_id
where a.order_status not in ('unavailable','canceled')
and a.order_purchase_timestamp is not null
""").to_df()

In [ ]:
df_customer_recency = duckdb.query("""
select b.customer_unique_id
, max(cast(order_purchase_timestamp as datetime)) lastest_txn
, min(cast(order_purchase_timestamp as datetime)) first_txn
, min(datediff('day', cast(order_purchase_timestamp as datetime), cast('2018-09-04' as datetime))) as day_after_last_pur
from df_order a
left join df_customer b on a.customer_id = b.customer_id
where a.order_status not in ('unavailable','canceled')
group by b.customer_unique_id
""").to_df()

In [ ]:
df_customer_recency.head()

5. frequency

In [ ]:
df_customer_frequency = duckdb.query("""
select b.customer_unique_id
, count(distinct order_id) no_order
from df_order a
left join df_customer b
on a.customer_id = b.customer_id
where a.order_status not in ('unavailable','canceled')
group by b.customer_unique_id
""").to_df()

In [ ]:
df_customer_frequency.head()

6. monetary

In [ ]:
df_customer_monetary = duckdb.query("""
select b.customer_unique_id
, sum(c.payment_value) total_amount
from df_order a
left join df_customer b
on a.customer_id = b.customer_id
left join df_order_payment c
on a.order_id = c.order_id
where a.order_status not in ('unavailable','canceled')
group by b.customer_unique_id
""").to_df()

In [ ]:
df_customer_monetary.head()

7. prefered category

In [ ]:
df_customer_preferred_cat = duckdb.query("""
select customer_unique_id
, count(distinct case when cat_new = 'home and kitchen' then product_id else null end)
/count(distinct product_id)*1.0 cat_home
, count(distinct case when cat_new = 'fashion and accessories' then product_id else null end)
/count(distinct product_id)*1.0 cat_fashion
, count(distinct case when cat_new = 'electronics and gadgets' then product_id else null end)
/count(distinct product_id)*1.0 cat_elec
, count(distinct case when cat_new = 'tools and construction' then product_id else null end)
/count(distinct product_id)*1.0 cat_tools
, count(distinct case when cat_new = 'health, beauty, and hygiene' then product_id else null end)
/count(distinct product_id)*1.0 cat_health_beauty
, count(distinct case when cat_new = 'books, music, and entertainment' then product_id else null end)
/count(distinct product_id)*1.0 cat_book_and_music
, count(distinct case when cat_new = 'toys and leisure' then product_id else null end)
/count(distinct product_id)*1.0 cat_toy
, count(distinct case when cat_new = 'miscellaneous' then product_id else null end)
/count(distinct product_id)*1.0 cat_others
from
    (
    select *
    , case when e.product_category_name_english in ('housewares', 'kitchen_dining_laundry_garden_furniture', 'home_confort', 'home_construction', 'home_appliances', 'home_appliances_2', 'bed_bath_table', 'furniture_bedroom', 'furniture_living_room', 'furniture_decor', 'furniture_mattress_and_upholstery', 'small_appliances', 'small_appliances_home_oven_and_coffee', 'garden_tools', 'home_comfort_2') then 'home and kitchen'
    when e.product_category_name_english in ('fashion_bags_accessories', 'fashion_male_clothing', 'fashion_sport', 'fashion_underwear_beach', 'fashion_childrens_clothes', 'fashion_shoes', 'fashio_female_clothing') then 'fashion and accessories'
    when e.product_category_name_english in ('electronics', 'fixed_telephony', 'consoles_games', 'audio', 'tablets_printing_image', 'computers_accessories', 'telephony', 'computers') then 'electronics and gadgets'
    when e.product_category_name_english in ('costruction_tools_garden', 'costruction_tools_tools', 'construction_tools_safety', 'construction_tools_lights', 'construction_tools_construction') then 'tools and construction'
    when e.product_category_name_english in ('health_beauty', 'diapers_and_hygiene') then 'health, beauty, and hygiene'
    when e.product_category_name_english in ('books_general_interest', 'books_imported', 'books_technical', 'music', 'cds_dvds_musicals', 'dvds_blu_ray', 'cine_photo') then 'books, music, and entertainment'
    when e.product_category_name_english in ('baby', 'sports_leisure', 'toys') then 'toys and leisure'
    else 'miscellaneous' end cat_new
    from df_order a
    left join df_customer b
    on a.customer_id = b.customer_id
    left join df_order_item c
    on a.order_id = c.order_id
    left join df_product d
    on c.product_id = d.product_id
    left join df_cat_name e
    on d.product_category_name = e.product_category_name
    where a.order_status not in ('unavailable','canceled')
    ) a
group by customer_unique_id
""").to_df()

In [ ]:
df_customer_preferred_cat.head()

8. prefered payment channel

In [ ]:
df_customer_preferred_payment = duckdb.query("""
select b.customer_unique_id
, count(distinct case when payment_type = 'credit_card' then a.order_id else null end)
/count(distinct a.order_id)*1.0 payment_credit
, count(distinct case when payment_type = 'debit_card' then a.order_id else null end) 
/count(distinct a.order_id)*1.0 payment_debit
, count(distinct case when payment_type = 'boleto' then a.order_id else null end)
/count(distinct a.order_id)*1.0 payment_ticket
, count(distinct case when payment_type = 'voucher' then a.order_id else null end)
/count(distinct a.order_id)*1.0 payment_voucher
from df_order a
left join df_customer b
on a.customer_id = b.customer_id
left join df_order_payment c
on a.order_id = c.order_id
where a.order_status not in ('unavailable','canceled')
group by b.customer_unique_id
""").to_df()

In [ ]:
df_customer_preferred_payment.head()

9. review score

In [ ]:
df_customer_review_score = duckdb.query("""
select b.customer_unique_id
, sum(c.review_score) avg_review_score
from df_order a
left join df_customer b
on a.customer_id = b.customer_id
left join df_order_review c
on a.order_id = c.order_id
where a.order_status not in ('unavailable','canceled')
group by b.customer_unique_id
""").to_df()

In [ ]:
df_customer_review_score.head()

# 3. Join data

In [ ]:
df = duckdb.query("""
select a.customer_unique_id
, b.age
, c.address_north, c.address_northeast, c.address_centralwest, c.address_southeast, c.address_south
, d.day_after_last_pur
, e.no_order
, f.total_amount
, g.cat_home, g.cat_fashion, g.cat_elec, g.cat_tools, g.cat_health_beauty
, g.cat_book_and_music, g.cat_toy, g.cat_others
, h.payment_credit, h.payment_debit, h.payment_ticket, h.payment_voucher
, i.avg_review_score

from df_customer_unique_id a
left join df_customer_age b
on a.customer_unique_id = b.customer_unique_id
left join df_customer_address c
on a.customer_unique_id = c.customer_unique_id
left join df_customer_recency d
on a.customer_unique_id = d.customer_unique_id
left join df_customer_frequency e
on a.customer_unique_id = e.customer_unique_id
left join df_customer_monetary f
on a.customer_unique_id = f.customer_unique_id
left join df_customer_preferred_cat g
on a.customer_unique_id = g.customer_unique_id
left join df_customer_preferred_payment h
on a.customer_unique_id = h.customer_unique_id
left join df_customer_review_score i
on a.customer_unique_id = i.customer_unique_id
""").to_df()

In [ ]:
df.head()

# 4. Handle missing value

In [ ]:
df.isna().sum()

In [ ]:
df['day_after_last_pur'].fillna(0, inplace=True)
df['no_order'].fillna(0, inplace=True)
df['total_amount'].fillna(0, inplace=True)
df['cat_home'].fillna(0, inplace=True)
df['cat_fashion'].fillna(0, inplace=True)
df['cat_elec'].fillna(0, inplace=True)
df['cat_tools'].fillna(0, inplace=True)
df['cat_health_beauty'].fillna(0, inplace=True)
df['cat_book_and_music'].fillna(0, inplace=True)
df['cat_toy'].fillna(0, inplace=True)
df['cat_others'].fillna(0, inplace=True)
df['payment_credit'].fillna(0, inplace=True)
df['payment_debit'].fillna(0, inplace=True)
df['payment_ticket'].fillna(0, inplace=True)
df['payment_voucher'].fillna(0, inplace=True)

In [ ]:
df['avg_review_score'].fillna(4.278182, inplace=True)

In [ ]:
df['day_after_last_pur'].fillna(1000, inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

# 5. Exploratory data analysis (EDA)

list of columns by types

In [ ]:
df.columns.tolist()

In [ ]:
id_col = ['customer_unique_id']

In [ ]:
cat_col = ['address_north','address_northeast','address_centralwest','address_southeast','address_south']

In [ ]:
num_col = ['age',
           'day_after_last_pur','no_order','total_amount',
           'cat_home','cat_fashion','cat_elec','cat_tools','cat_health_beauty',
           'cat_book_and_music','cat_toy','cat_others',
           'payment_credit','payment_debit','payment_ticket','payment_voucher',
           'avg_review_score']

categorical features

In [ ]:
df[cat_col].describe()

numeric feature

In [ ]:
for column_name in num_col:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.hist(df[column_name], bins=10, edgecolor='black', alpha=0.7)
    ax1.set_title(f'Histogram of {column_name}')
    ax1.set_xlabel(column_name)
    ax1.set_ylabel('Frequency')
    ax1.grid(axis='y', alpha=0.75)
    
    if isinstance(df[column_name].dtype, pd.CategoricalDtype):
        sns.boxplot(x=df[column_name], ax=ax2)
    else:
        sns.boxplot(x=df[column_name], ax=ax2)
    ax2.set_title(f'Boxplot of {column_name} (No Hue)')
    ax2.set_xlabel(column_name)
    
    plt.show()

drop unnecessary columns

In [ ]:
df.drop(columns = 'customer_unique_id', inplace = True)

correlation

In [ ]:
corr = df.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.heatmap(corr, annot=True, ax=ax)

# 6. Model

In [ ]:
df.head()

standardize 

In [ ]:
df = df[cat_col + num_col]

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(df)

In [ ]:
df_scaled = pd.DataFrame(scaler.transform(df), columns=cat_col + num_col)

In [ ]:
df

In [ ]:
df_scaled

PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
df_pca = pd.DataFrame(data=pca.fit_transform(df_scaled), columns=['Principal Component 1', 'Principal Component 2'])

In [ ]:
df_pca

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
plt.scatter(df_pca['Principal Component 1'], df_pca['Principal Component 2'], c='blue', label='Data Points')

plt.title('PCA Results with 2 Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()

plt.show()

elbow method

In [ ]:
range_n_clusters = range(2, 11)

In [ ]:
wcss = []

In [ ]:
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(df_pca)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range_n_clusters, wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

silhouette

In [ ]:
silhouette_scores = []

In [ ]:
for n_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(df_pca)
    silhouette_avg = silhouette_score(df_pca, cluster_labels)
    silhouette_scores.append(silhouette_avg)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.title('Silhouette Scores')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

KMeans (k=4)

In [ ]:
kmeans_k4 = KMeans(n_clusters=4, random_state=42)

In [ ]:
kmeans_k4.fit(df_pca)

In [ ]:
clusters_kmean_k4 = kmeans_k4.predict(df_pca)

In [ ]:
clusters_kmean_k4

In [ ]:
df_pca['cluster (Kmean k=4)'] = clusters_kmean_k4
df['cluster (Kmean k=4)'] = clusters_kmean_k4

KMeans (k=6)

In [ ]:
kmeans_k6 = KMeans(n_clusters=6, random_state=42)

In [ ]:
kmeans_k6.fit(df_pca)

In [ ]:
clusters_kmean_k6 = kmeans_k6.predict(df_pca)

In [ ]:
clusters_kmean_k6

In [ ]:
df_pca['cluster (Kmean k=6)'] = clusters_kmean_k6
df['cluster (Kmean k=6)'] = clusters_kmean_k6

Plot

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_pca['Principal Component 1'], df_pca['Principal Component 2'], c=df_pca['cluster (Kmean k=4)'], cmap='viridis', label='Cluster Label')

plt.title('PCA Results with 2 Components and K-Means Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_pca['Principal Component 1'], df_pca['Principal Component 2'], c=df_pca['cluster (Kmean k=6)'], cmap='viridis', label='Cluster Label')

plt.title('PCA Results with 2 Components and K-Means Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

Average value by cluster

In [ ]:
df.groupby('cluster (Kmean k=4)').mean().T

In [ ]:
df.groupby('cluster (Kmean k=6)').mean().T

In [ ]:
***

# Save model

In [ ]:
with open('saved clustering model/scaler_model.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

In [ ]:
with open('saved clustering model/kmeans_k4_model.pkl', 'wb') as kmeans_file:
    pickle.dump(kmeans_k4, kmeans_file)